In [ ]:
from yahoo_oauth import OAuth2
import os
import json
import league, game, team
import yhandler
import json
import objectpath
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# get our session context ("sc")
oauth = OAuth2(None, None, from_file='oauth2.json')

[2020-09-02 14:38:42,468 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2020-09-02 14:38:42,472 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 77223.14102721214
[2020-09-02 14:38:42,474 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN HAS EXPIRED
[2020-09-02 14:38:42,475 DEBUG] [yahoo_oauth.yahoo_oauth.refresh_access_token] REFRESHING TOKEN


In [3]:
# prepare functions
def get_json_dump(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return json.dumps(jresp)
def get_json(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return jresp
def get_json_t(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    t = objectpath.Tree(jresp)
    return t

In [4]:
gm = game.Game(oauth, 'nfl')

#use package to get league_id
league_ids = gm.league_ids()
print(league_ids)
test_league_id = league_ids[2]
print(test_league_id)

# joesleagueid = '390.l.974698'
# steve's league: 390.l.707700

['390.l.707700', '390.l.974698', '399.l.242160']
399.l.242160


In [7]:
# only get leagues from this year
gm.league_ids(2020) 

['399.l.242160']

In [12]:
# create league from game
league2020 = gm.to_league('390.l.707700')
league2020

In [13]:
league2020.teams()

[{'name': 'Resident Stevil', 'team_key': '390.l.707700.t.1'},
 {'name': 'War Machine', 'team_key': '390.l.707700.t.10'},
 {'name': 'Enemies of Carlotta', 'team_key': '390.l.707700.t.9'},
 {'name': 'Hawkeyes', 'team_key': '390.l.707700.t.4'},
 {'name': 'KnightHawks', 'team_key': '390.l.707700.t.6'},
 {'name': 'Wedgie Woty', 'team_key': '390.l.707700.t.12'},
 {'name': 'Dude in Omaha', 'team_key': '390.l.707700.t.2'},
 {'name': 'Menty', 'team_key': '390.l.707700.t.7'},
 {'name': 'Healthy Scratch', 'team_key': '390.l.707700.t.5'},
 {'name': 'Hair Force Blue', 'team_key': '390.l.707700.t.3'},
 {'name': 'Werkmonsters', 'team_key': '390.l.707700.t.11'},
 {'name': 'Monarchs', 'team_key': '390.l.707700.t.8'}]

In [14]:
league2020.team_key()
# healthy scratch: '390.l.707700.t.5'

'390.l.707700.t.5'

In [ ]:
# def get_players_by_status(self, league_id, status):
#         """Return the raw JSON when requesting standings for a league.

#         :param league_id: League ID to get the standings for
#         :type league_id: str
#         :return: JSON document of the request.
#         """
#         return self.get("league/{}/players".format(league_id))


In [ ]:
###Player RESOURCE
# https://fantasysports.yahooapis.com/fantasy/v2/player/ 

#get a player by player key
playersurl = "league/390.l.974698/players;player_keys=390.p.5479"
player = get_json(playersurl)
player

In [ ]:
#get a player's stats'
statsurl = "league/390.l.974698/players;player_keys=390.p.5479/stats"
stats = get_json(statsurl)
stats

In [ ]:
# players COLLECTION
qburl = "league/390.l.974698/players;position=QB"
qbs = get_json_t(qburl)
qbs

In [ ]:
qbt = get_json_t(qburl)
jfilter = qbt.execute('$..(player)')
qbs = []
for row in jfilter:
#     print(row)
#     print("-----------------------------------------")
#     print("---------------nextrow-------------------")
#     print("-----------------------------------------")
    qbs.append(row)
qbs[0]


In [ ]:
qbs[0]['player'][0]

In [ ]:
# combine into one dictionary
firstqb = qbs[0]['player'][0]
onelist = [item for item in firstqb if item != []]
onelist
onetdict = {k: v for d in onelist for k, v in d.items()}
onetdict

In [ ]:
raw_qb_data = json_normalize(onetdict, sep="_")
raw_qb_data

In [ ]:
all_qb_list = []
allqbs = qbs
allqbs

In [ ]:
# for k, v in allqbs.items():
#     if k != "count": #ignore count item which json adds as autosummary at the end
#         qb = allqbs[k]['player'][0] #go down to individual player level in json
#         qblist = [item for item in qb if item != []]     #remove all empty lists
#         qbdict = { k : v[0] if type(v)==list else v  for d in qblist for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
#         print(qbdict)
#         all_qb_list.append(qbdict)

for item in allqbs:
#     print(item)
    qb = item['player'][0]
    qblist = [item for item in qb if item != []]     #remove all empty lists
    qbdict = { k : v[0] if type(v)==list else v  for d in qblist for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
#     print(qbdict)
    all_qb_list.append(qbdict)
print(all_qb_list)
#     if k != "count": #ignore count item which json adds as autosummary at the end
#         qb = allqbs[k]['player'][0] #go down to individual player level in json
#         qblist = [item for item in qb if item != []]     #remove all empty lists
#         qbdict = { k : v[0] if type(v)==list else v  for d in qblist for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
#         print(qbdict)
#         all_qb_list.append(qbdict)

In [ ]:
qbsdf = json_normalize(all_qb_list, sep="_") #this only gets the sub rows?
qbsdf

In [ ]:
# create universal function for turning API data from a player into a dataframe
positionurl = "league/390.l.974698/players;position=TE"
position_t = get_json_t(positionurl)
positionfilter = position_t.execute('$..(player)')


position_player_list = []
for row in positionfilter:
    position_player_list.append(row)
position_player_list

In [ ]:
# all_players = []
# for item in position_player_list:
#     print("item")
#     print(item)
#     player = item['player'][0]
#     print("player")
#     print(player)
#     temp_list = [item for item in player if item != []]     #remove all empty lists
# #     print("temp list")
# #     print(temp_list)
#     position_player_dict = { k : v[0] if type(v)==list else v  for d in temp_list for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
#     print("ppd")
#     print(position_player_dict)
#     all_players.append(position_player_dict)

In [ ]:
    
# player_position_df = json_normalize(position_player_list, sep="_") #this only gets the sub rows?
# player_position_df

In [ ]:
# # players COLLECTION
# qburl2 = "league/390.l.974698/players;position=TE"

# qbt2 = get_json_t(qburl2)
# jfilter2 = qbt2.execute('$..(player)')
# qbs2 = []
# for row in jfilter2:
#     qbs2.append(row)
# # qbs2[0]
# # qbs2[0]['player'][0]
# all_qb_list2 = []
# allqbs2 = qbs2
# allqbs2
# for item in allqbs2:
# #     print(item)
#     qb = item['player'][0]
#     qblist = [item for item in qb if item != []]     #remove all empty lists
#     qbdict = { k : v[0] if type(v)==list else v  for d in qblist for k, v in d.items()} 
# #     print(qbdict)
#     all_qb_list2.append(qbdict)
# # print(all_qb_list)
# qbsdf2 = json_normalize(all_qb_list2, sep="_") #this only gets the sub rows?
# qbsdf2

In [ ]:
def get_position_df(position_abbreviation):
    # players COLLECTION
    url = "league/390.l.974698/players;position=" + position_abbreviation + ";count=100"
    print(url)
    tree = get_json_t(url)
    jfiltert = tree.execute('$..(player)')
    results = []
    for row in jfiltert:
        results.append(row)
        # qbs2[0]['player'][0]
    results_list = []
    all_player_position_list = []
    for item in results:
        position_player = item['player'][0]
        position_player_list = [item for item in position_player if item != []]     #remove all empty lists
        qbdict = { k : v[0] if type(v)==list else v  for d in position_player_list for k, v in d.items()} 
        all_player_position_list.append(qbdict)
    position_df = json_normalize(all_player_position_list, sep="_")
    return position_df

In [ ]:
te_df = get_position_df("TE")
te_df

In [ ]:
wr_df = get_position_df("WR")
wr_df

In [ ]:
# need to figure out why only retrieving 25 rows each time
rb_df = get_position_df("RB")
rb_df